## Visualize model outputs

#### Disabling Jupyter's inline scroll option.

In [ ]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

#### Reading input files
Set ```working_path``` output "images" folder

In [ ]:
import os

working_path = "/home/nagellette/Desktop/model_outputs/unet_2021_01_14-21_18_40/images/"

file_list = os.listdir(working_path)
print("Working path: {}".format(working_path))
print("Read {} files.".format(len(file_list)))

#### Process input files to avoid the ones that have data but not prediction

In [ ]:
limit = 0
for i in range(len(file_list)):
    for file in file_list:
        if file.startswith("{}_".format(str(i))) and "predict" in file:
            print(file)
            limit += 1
            break
            
print("Number of prediction examples: {}".format(limit))

#### Read predictions, labels and Sentinel RGB images.
- ```threshold```set to 0.5, change if required.
- Known issues:
    - Sentinel RGB image seems false color due to preprocessing.
    - Sentinel B2, B3, B4 bands are saved as B1, B2, B3.


In [ ]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import matplotlib

%matplotlib inline

matplotlib.rcParams['figure.figsize'] = [20, 20]

for i in range(limit):
    res = [idx for idx in file_list if idx.startswith("{}_".format(i))]
    f,ax = plt.subplots(1,3, gridspec_kw={'width_ratios': [10, 10, 10]})
    for file in res:
        if "predict" in file and "_2." in file:
            predict = mpimg.imread('{}/{}'.format(working_path, file))
            predict = predict > 0.5
            predict = predict.astype(int)
            ax[0].imshow(predict, cmap="gray")
            ax[0].set_title("Prediction")
        
        elif "B02" in file:
            # blue
            blue = mpimg.imread('{}/{}'.format(working_path, file))
            print(np.max(blue))
        
        elif "B03" in file:
            # green
            green = mpimg.imread('{}/{}'.format(working_path, file))
        
        elif "B04" in file:
            # red
            red = mpimg.imread('{}/{}'.format(working_path, file))
        
        elif "label" in file and "_0_0." in file:
            # label
            label = mpimg.imread('{}/{}'.format(working_path, file))
            ax[1].imshow(label, cmap="gray")
            ax[1].set_title("Label")
            
    sentinel_img = np.dstack((red, green, blue))
    ax[2].imshow(sentinel_img, cmap="magma")
    ax[2].set_title("Sentinel")
  
    for x in ax:
        x.set_xticks([])
        x.set_yticks([])
    
    plt.show()